In [1]:
import torch
import torch.nn as nn

In [2]:
import wandb
!wandb login

wandb: Currently logged in as: rajceo2031 (rentio) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
from dataclasses import dataclass


@dataclass 
class ModelArgs:
    device = 'cuda'
    no_of_neurons = 128
    block_size = 64
    batch_size = 32
    dropout = 0.1
    epoch = 50
    max_lr = 1e-4

In [4]:
torch.set_default_device(ModelArgs.device)

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader


torch.manual_seed(0)


num_samples = 1000  
seq_length = ModelArgs.block_size  
device = ModelArgs.device  


t = torch.linspace(0, 100, num_samples + seq_length, device=device)
data = torch.sin(t) + 0.1 * torch.randn_like(t)  


X_tensor = torch.stack([data[i:i+seq_length] for i in range(num_samples)])
y_tensor = data[seq_length:]  # Next value prediction

train_size = int(0.8 * num_samples)

X_train, y_train = X_tensor[:train_size], y_tensor[:train_size]  
X_val, y_val = X_tensor[train_size:], y_tensor[train_size:]  


class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


train_dataset = TimeSeriesDataset(X_train, y_train)
val_dataset = TimeSeriesDataset(X_val, y_val)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


generator = torch.Generator(device=device)


train_loader = DataLoader(
    train_dataset,
    batch_size=ModelArgs.batch_size,
    shuffle=True,  
    generator=generator,  
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    drop_last=True,
    batch_size=ModelArgs.batch_size,
    shuffle=False, 
)



In [6]:
class ResetGate(torch.nn.Module):
    def __init__(self, device, no_of_neurons):
        super().__init__()
        self.linear = nn.Linear(in_features=ModelArgs.block_size + ModelArgs.no_of_neurons, out_features=no_of_neurons, device=device, dtype=torch.float32)
   
    def forward(self, x, ht_1):
        x = torch.cat([x, ht_1], dim=-1)
        out = torch.nn.functional.sigmoid(self.linear(x))
        return out

In [7]:
class UpdateGate(nn.Module):
    def __init__(self, device, no_of_neurons) -> None:
        super().__init__()
        self.linear = nn.Linear(in_features=ModelArgs.block_size + ModelArgs.no_of_neurons, out_features=no_of_neurons, device=device, dtype=torch.float32)
    def forward(self, x, ht_1):
        x = torch.cat([x, ht_1], dim=-1)
        out = torch.nn.functional.sigmoid(self.linear(x))
        return out

In [12]:
class GRUBlock(nn.Module):
    def __init__(self, device, no_of_neurons) -> None:
        super().__init__()
        self.update = UpdateGate(device=device, no_of_neurons=no_of_neurons)
        self.reset = ResetGate(device=device, no_of_neurons=no_of_neurons)
        self.candidate = nn.Linear(in_features=ModelArgs.no_of_neurons, out_features=no_of_neurons, device=device, dtype=torch.float32)
        
    def forward(self, x, ht_1):
        reset_out = self.reset(x, ht_1)
        ht_1_candidate = nn.functional.tanh(self.candidate(reset_out))
        update_out = self.update(x, ht_1)
        ht = ((1- update_out) * ht_1) + (update_out * ht_1_candidate)
        return ht

In [20]:
class GRU(nn.Module):
    def __init__(self, device, no_of_neurons, out_features):
        super().__init__()
        self.block1 = GRUBlock(device=device, no_of_neurons=no_of_neurons)
        self.block2 = GRUBlock(device=device, no_of_neurons=no_of_neurons)
        self.ht_1 = torch.randn(ModelArgs.batch_size, no_of_neurons, device=device, requires_grad=True, dtype=torch.float32)
        # self.ct_1 = torch.randn(ModelArgs.batch_size, no_of_neurons,device=device, requires_grad=True, dtype=torch.float32)
        self.output = nn.Linear(in_features=ModelArgs.no_of_neurons, out_features=out_features, device=device, dtype=torch.float32)
        self.dropout = nn.Dropout(p=ModelArgs.dropout)
        # self.embedding = nn.Embedding()
        
    def forward(self, x):
        # x = 
        # print("LSTM: ",x.shape)
        # print("LSTM: ", self.ht_1.shape)
        # print("LSTM: ", self.ct_1.shape)
        ht = self.block1(x, self.ht_1)
        # print(ht.shape)
        # print(ct.shape)
        ht = self.block2(x, ht)
        ht = self.dropout(ht)
        # print("After: ", ht.shape)
        out = self.output(ht)
        return out

In [21]:
model = GRU(device=ModelArgs.device, no_of_neurons=ModelArgs.no_of_neurons, out_features=1)
model = model.to(ModelArgs.device)

In [ ]:
!pip install torchinfo

from torchinfo import summary

x = torch.randint(0, 100, (ModelArgs.batch_size,ModelArgs.block_size))  # Random integer between 0 and 100
x = x.to(ModelArgs.device)
summary(model=model,
        input_data=x,
        # input_size=(ModelArgs.batch_size, ModelArgs.block_size, ModelArgs.embeddings_dims),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
GRU (GRU)                                [32, 64]             [32, 1]              --                   True
├─GRUBlock (block1)                      [32, 64]             [32, 128]            --                   True
│    └─ResetGate (reset)                 [32, 64]             [32, 128]            --                   True
│    │    └─Linear (linear)              [32, 192]            [32, 128]            24,704               True
│    └─Linear (candidate)                [32, 128]            [32, 128]            16,512               True
│    └─UpdateGate (update)               [32, 64]             [32, 128]            --                   True
│    │    └─Linear (linear)              [32, 192]            [32, 128]            24,704               True
├─GRUBlock (block2)                      [32, 64]             [32, 128]            --                   True
│    └─ResetGa

In [22]:
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=ModelArgs.max_lr)

In [ ]:
model.train()
train_losses =  torch.zeros(len(train_loader))
val_losses = torch.zeros(len(val_loader))
wandb.init(
    project='GRU-From-Scratch'
)
for epoch in range(ModelArgs.epoch):
    
    count = 0
    for X, y in train_loader:
        y_pred = model(X)
        # print(y_pred.shape)
        loss = criterion(y_pred, y)
        train_losses[count] = loss.item()
        # print("Loss: ", loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1
        
        wandb.log({
      "Train Loss": loss.item()
      "Val Loss": loss.item(),
      "step": count  
    })
    # count = 0
    model.eval()
    count = 0
    for X, y in val_loader:
        y_pred = model(X)
        # print(y_pred.shape)
        loss = criterion(y_pred, y)
        
        # print("Loss: ", loss.item())
        val_losses[count] = loss.item()
        
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        count += 1
        wandb.log({
      "Train Loss": loss.item(),
      "Val Loss": loss.item(),
      "step": count  
    })
    model.train()
    wandb.log({
      "Train Loss": train_losses.mean(),
      "Val Loss": val_losses.mean(),
      "epoch": epoch  
    })
    print("Epoch: ", epoch, "|", "Train Loss: ", train_losses.mean(),  "|", "Val Loss: ", val_losses.mean())


wandb: Currently logged in as: rajceo2031 (rentio) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/home/yuvrajsingh/anaconda3/envs/unsloth_env/lib/python3.11/site-packages/torch/utils/_device.py:106: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return func(*args, **kwargs)


Epoch:  0 | Train Loss:  